In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import glob

# -----------------------------
# 📥 Step 1: Load economic + population data
# -----------------------------
eco_df = pd.read_csv("new-eco.csv")
pop_df = pd.read_excel("onlypopulation-cleaned.xlsx")

# Strip and unify column names
eco_df.columns = eco_df.columns.map(lambda x: str(x).strip())
pop_df.columns = pop_df.columns.map(lambda x: str(x).strip())

# Rename for consistency
if "SA2 Name" in eco_df.columns:
    eco_df.rename(columns={"SA2 Name": "SA2_Name"}, inplace=True)
if "SA2 Name" in pop_df.columns:
    pop_df.rename(columns={"SA2 Name": "SA2_Name"}, inplace=True)

# Fill in fake SA2_Name if missing
if "SA2_Name" not in eco_df.columns:
    eco_df["SA2_Name"] = "UNKNOWN"
if "SA2_Name" not in pop_df.columns:
    pop_df["SA2_Name"] = "UNKNOWN"

# Merge economic + population data
merged_df = pd.merge(eco_df, pop_df, on="SA2_Name", how="inner")

# -----------------------------
# 🚆 Step 2: Process Opal patronage files safely
# -----------------------------
opal_files = glob.glob("Opal_Patronage_*.txt")
opal_grouped_list = []

for file in opal_files:
    try:
        df = pd.read_csv(file, delimiter="|", usecols=["ti_location_name", "trip_count"], engine="python", low_memory=False)
        df.columns = df.columns.map(lambda x: str(x).strip())
        df = df.dropna(subset=["ti_location_name", "trip_count"])
        df["trip_count"] = pd.to_numeric(df["trip_count"], errors="coerce").fillna(0)
        grouped = df.groupby("ti_location_name")["trip_count"].sum().reset_index()
        opal_grouped_list.append(grouped)
    except Exception as e:
        print(f"⚠️ Skipped {file}: {e}")

# Final combined trips
if opal_grouped_list:
    opal_grouped = pd.concat(opal_grouped_list, ignore_index=True)
    opal_grouped = opal_grouped.groupby("ti_location_name")["trip_count"].sum().reset_index()
    opal_grouped.rename(columns={"ti_location_name": "SA2_Name", "trip_count": "Total_Trips"}, inplace=True)
else:
    opal_grouped = pd.DataFrame(columns=["SA2_Name", "Total_Trips"])

# -----------------------------
# 🔁 Step 3: Final merge
# -----------------------------
final_df = pd.merge(merged_df, opal_grouped, on="SA2_Name", how="left")
final_df["Total_Trips"].fillna(0, inplace=True)

# -----------------------------
# 📊 Step 4: Plot and correlation
# -----------------------------
if "Pop_Growth_Percent" not in final_df.columns:
    final_df["Pop_Growth_Percent"] = 0

sns.scatterplot(data=final_df, x="Pop_Growth_Percent", y="Total_Trips")
plt.title("Population Growth vs Opal Trips")
plt.xlabel("Population Growth (%)")
plt.ylabel("Total Opal Trips")
plt.grid(True)
plt.show()

correlation = final_df["Pop_Growth_Percent"].corr(final_df["Total_Trips"])
print(f"🔗 Correlation: {correlation:.3f}")

# -----------------------------
# 🤖 Step 5: Clustering
# -----------------------------
features = final_df[["Pop_Growth_Percent", "Total_Trips"]].fillna(0)
scaled = StandardScaler().fit_transform(features)

kmeans = KMeans(n_clusters=3, random_state=42)
final_df["Cluster"] = kmeans.fit_predict(scaled)

sns.scatterplot(data=final_df, x="Pop_Growth_Percent", y="Total_Trips", hue="Cluster", palette="viridis")
plt.title("Clustering SA2 Areas by Growth & Transport")
plt.xlabel("Population Growth (%)")
plt.ylabel("Total Opal Trips")
plt.grid(True)
plt.show()
